<a href="https://colab.research.google.com/github/ffedox/pbr/blob/main/corpus_creation_with_comments_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting a domain-specific parallel corpus from Wikipedia

In [1]:
import torch

# If there's a GPU avaiable, tell PyTorch to use the GPU,
# otherwise, using the CPU instead.
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('Found GPU:', torch.cuda.get_device_name(0))
else:
  device = torch.device("cpu")
  print('CPU will be used because no GPU available.')

Found GPU: Tesla T4


# 1. Setup

## 1.1 Wikipedia API

[Wikipedia-API](https://github.com/martin-majlis/Wikipedia-API) is a Python wrapper for Wikipedias' API. It supports extracting texts, sections, links, categories, translations, etc from Wikipedia.

In [2]:
!pip install wikipedia-api --quiet

## 1.2 Sentence Transformers



[Sentence Transformers](https://www.sbert.net/) is a Python framework for state-of-the-art sentence, text and image embeddings. 

In [3]:
!pip install sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.0 MB/s eta 0:00:00


## 1.3 Imports

In [4]:
from sentence_transformers import models, SentenceTransformer
from json import JSONDecodeError

import wikipediaapi

import pandas as pd
import numpy as np
import scipy

from nltk.tokenize import sent_tokenize
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 2. Extracting an IT-EN comparable corpus from Category: pages

First we will extract a comparable corpus by scraping Category: pages in Italian (it is more likely to find correspondences from IT to EN than the other way around).

For each page linked in the IT Category: page, we will be retrieving the EN equivalent.

*Here, we will limit the search to the summaries (first paragraph in each Wikipedia entry).* Maybe not?

## 2.1 Extracting the IT articles from the Category

In [5]:
page_titles = []
page_texts_it = []

def get_it_articles(category, page_titles, page_texts_it):

  wiki_wiki = wikipediaapi.Wikipedia('it')
  cat = wiki_wiki.page(category)   # category should be like .page("Categoria:Survival_horror")

  for p in cat.categorymembers.values():
    if p.namespace == wikipediaapi.Namespace.MAIN:
      # it is page => we can get text
      page_titles.append(p.title)
      page_texts_it.append(p.text)

In [6]:
get_it_articles("Categoria:Videogiochi_d'avventura", page_titles, page_texts_it)

## 2.2 Extracting the corresponding EN articles

By leveraging the previously obtained titles, we can search for the equivalent pages on the EN Wikipedia.

In [7]:
page_texts_en = []

def get_langlinks(page, page_texts_en):

        langlinks = page.langlinks

        for k in sorted(langlinks.keys()):
            v = langlinks[k]

        try:
          page_en = page.langlinks['en']
          page_texts_en.append(page_en.text)

        except KeyError:
          page_texts_en.append(str('No match'))
          
        except JSONDecodeError:
           page_texts_en.append(str('No match'))

In [8]:
def get_en_articles(page_titles, page_texts_en):

  for title in page_titles:

    wiki_wiki = wikipediaapi.Wikipedia('it')
    page = wiki_wiki.page(str(title))
    get_langlinks(page, page_texts_en)

In [9]:
get_en_articles(page_titles, page_texts_en)

## 2.3 Comparable IT-EN corpus

We merge the IT and the EN pages to obtain a comparable corpus.

In [10]:
comparable_corpus_vg = pd.DataFrame(np.column_stack([page_texts_en, page_texts_it]), 
                               columns=['en', 'it'])

Now we only need to drop the rows where no English equivalent was found.

In [11]:
comparable_corpus_vg = comparable_corpus_vg[comparable_corpus_vg['en'].str.contains('No match')==False].reset_index(drop=True)

End result:

In [12]:
comparable_corpus_vg

,en,it
0,13 Sentinels: Aegis Rim is a 2019 video game d...,13 Sentinels: Aegis Rim (十三機兵防衛圏?) è un videog...
1,428: Shibuya Scramble is a visual novel advent...,428: Shibuya Scramble (428 封鎖された渋谷で 428: Fūsa ...
2,A Bird Story is an adventure role-playing vide...,A Bird Story è un videogioco d'avventura con e...
3,A Short Hike is an adventure video game by Can...,A Short Hike è un videogioco d'avventura del 2...
4,Ace Attorney Investigations: Miles Edgeworth i...,Ace Attorney Investigations: Miles Edgeworth (...
...,...,...
364,Yakuza Kiwami 2 is an action-adventure video g...,"Yakuza Kiwami 2, pubblicato in Giappone come R..."
365,Yume Nikki is a 2004 adventure game created by...,"Yume nikki (ゆめにっき? lett. ""Diario dei sogni"") è..."
366,"Mansion of Hidden Souls, called Tale of the Dr...",Yumemi Mystery Mansion (夢見館の物語 Yumemi yakata n...
367,"Famicom Mukashibanashi: Yūyūki (ふぁみこんむかし話 遊遊記,...",Yūyūki (遊遊記?) è un videogioco d'avventura del ...


# 3. Extracting parallel sentences from the comparable corpus

For each Wikipedia article, we obtained the link to the corresponding article in English. This can be used to mine sentences limited to the respective articles. This local approach has several advantages: 

1.   Mining is faster since each article usually has a few hundreds of sentences only.
2.   It seems reasonable to assume that a translation of a sentence is more
likely to be found in the same article than anywhere in the whole Wikipedia. 

## 3.1 Loading the Sentence-Transformers model

Sentence-BERT (SBERT) is a modification of the pretrained BERT network that uses siamese and triplet network structures to derive semantically meaningful sentence embeddings that can be compared using cosine-similarity [[1]](https://arxiv.org/abs/1908.10084://)

We will be using [distiluse-base-multilingual-cased-v2](https://huggingface.co/sentence-transformers/distiluse-base-multilingual-cased-v2://) to compute the sentence embeddings for evaluating textual similarity.

In [ ]:
model = SentenceTransformer('distiluse-base-multilingual-cased', device='cuda')

## 3.2 Using the model to create the embeddings, and computing sentence similarity

In [14]:
parallel_sentences_en = []
parallel_sentences_it = []

def extract_parallel_sents(comparable_corpus_vg, parallel_sentences_en, parallel_sentences_it):

  closest_n = 1

  for en, it in zip(comparable_corpus_vg.en.values, comparable_corpus_vg.it.values): # Looping throuth the texts in the comparable corpus

    corpus = sent_tokenize(en)
    queries = sent_tokenize(it)

    corpus_embeddings = model.encode(corpus)
    query_embeddings = model.encode(queries)

    for query, query_embedding in zip(queries, query_embeddings):
      
      distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

      results = zip(range(len(distances)), distances)
      results = sorted(results, key=lambda x: x[1])

      for idx, distance in results[0:closest_n]:
        if 1-distance > 0.8: # Similarity threshold
          parallel_sentences_it.append(query)
          parallel_sentences_en.append(corpus[idx].strip())

In [15]:
extract_parallel_sents(comparable_corpus_vg, parallel_sentences_en, parallel_sentences_it)

## 3.3 Building the parallel corpus 

In [16]:
parallel_corpus = pd.DataFrame(np.column_stack([parallel_sentences_en, parallel_sentences_it]), 
                               columns=['en', 'it'])

In [26]:
parallel_corpus

,en,it
0,Astro Bot Rescue Mission is a 2018 platform vi...,Astro Bot Rescue Mission è un videogioco a pia...
1,It stars a cast of robot characters first intr...,I robottini presenti nel gioco sono stati intr...
2,Gameplay\nAstro Bot Rescue Mission is a 3D pla...,Modalità di gioco\nAstro Bot Rescue Mission è ...
3,"Astro is able to jump, hover, punch and charge...","Astro è in grado di saltare, caricare e colpir..."
4,There are 8 lost robots in each level and find...,Ci sono 8 robot smarriti in ogni livello e tro...
...,...,...
193,Gameplay\nStar Wars: Squadrons is a space comb...,Star Wars: Squadrons è un videogioco di combat...
194,"As players earn more experience, they can unlo...",È anche possibile ottenere esperienza in modo ...
195,"An updated arcade version, Tekken 7: Fated Ret...",Tekken 7 è uscito nelle sale giochi giapponesi...
196,Until Dawn: Rush of Blood is a rail shooter de...,Until Dawn: Rush of Blood è un videogioco di g...


In [17]:
parallel_corpus.to_excel("parallel_corpus_vg_en_it_3.xlsx")  